# Inference 101: Make a Prediction

Simple example: Load one customer record and get a prediction from the model.

In [40]:
import pandas as pd
import json
import requests
import numpy as np
import sys
import os

# Add module1 to path for imports
sys.path.insert(0, '/home/cdsw/module1')

print("Libraries imported")

Libraries imported


**Get this from your model endpoint**

In [41]:
# Model endpoint
MODEL_ENDPOINT = "https://modelservice.ml-dbfc64d1-783.go01-dem.ylcu-atmi.cloudera.site/model"
ACCESS_KEY = "mbtbh46x9h7wxj4cdkxz9fxl0nzmrefv"

print(f"Model: {MODEL_ENDPOINT}")

Model: https://modelservice.ml-dbfc64d1-783.go01-dem.ylcu-atmi.cloudera.site/model


## Step 1: Load Single Row of Preprocessed Data

In [42]:
# Load and preprocess data following the same process as test_endpoint.py
from helpers import PreprocessingPipeline, FeatureEngineer


# Load raw inference data
df_raw = pd.read_csv("inference_data/raw_inference_data.csv", sep=";")
print(f"Loaded raw data: {df_raw.shape}")

# Load full training data for preprocessing fit
df_train = pd.read_csv("data/bank-additional/bank-additional-full.csv", sep=";")
print(f"Loaded training data: {df_train.shape}")

# Apply feature engineering
fe = FeatureEngineer()
df_train_eng = fe.transform(df_train)
df_engineered = fe.transform(df_raw)

# Define features
numeric_features = [
    'age', 'duration', 'campaign', 'pdays', 'previous',
    'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
    'engagement_score'
]

categorical_features = [
    'job', 'marital', 'education', 'default',
    'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome',
    'age_group', 'emp_var_category', 'duration_category'
]

# Create and fit preprocessor on full training data
preprocessor = PreprocessingPipeline(
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    include_engagement=True
)

X_train_full = df_train_eng[numeric_features + categorical_features].copy()
preprocessor.fit(X_train_full)

# Transform inference data
X_engineered = df_engineered[numeric_features + categorical_features].copy()
X_processed = pd.DataFrame(
    preprocessor.transform(X_engineered),
    columns=preprocessor.get_feature_names()
)



Loaded raw data: (1000, 20)
Loaded training data: (41188, 21)


In [43]:
# Pick a row number :

selected_row = 3

single_row = X_processed.iloc[[selected_row]]

print(f"Loaded and preprocessed single row")
print(f"Shape: {single_row.shape}")
print(f"Columns: {single_row.shape[1]}")

Loaded and preprocessed single row
Shape: (1, 64)
Columns: 64


## Step 2: Format for API

In [44]:
# Create the API payload with proper handling of NaN/inf values
row_dict = single_row.iloc[0].to_dict()

# Convert NaN and inf values for JSON serialization
for key, val in row_dict.items():
    if pd.isna(val):
        row_dict[key] = 0
    elif np.isinf(val):
        row_dict[key] = 1e10 if val > 0 else -1e10
    else:
        row_dict[key] = float(val)

payload = {
    "accessKey": ACCESS_KEY,
    "request": {
        "dataframe_split": {
            "columns": list(row_dict.keys()),
            "data": [list(row_dict.values())]
        }
    }
}

payload_json = json.dumps(payload)

print(f"Payload created")
print(f"Size: {len(payload_json)} bytes")

Payload created
Size: 1766 bytes


## Step 3: Send to Model Service

In [45]:
# Send request to model
response = requests.post(
    MODEL_ENDPOINT,
    data=payload_json,
    headers={'Content-Type': 'application/json'},
    timeout=30
)

print(f"Response status: {response.status_code}")

Response status: 403


## Step 4: Get the Prediction

In [39]:
# Parse response and handle errors
print(f"Response status: {response.status_code}\n")

if response.status_code != 200:
    print("❌ Error in response!")
    print(f"Status: {response.status_code}")
    print(f"Response text: {response.text}")
else:
    try:
        result = response.json()
        prediction = result['response']['prediction'][0]

        print(f"\n" + "="*50)
        print(f"PREDICTION: {prediction}")
        print(f"="*50)

        if prediction == 0:
            print("\nResult: Customer will NOT subscribe to term deposit")
        else:
            print("\nResult: Customer WILL subscribe to term deposit")

        print(f"\nFull response:")
        print(result)
        
    except (KeyError, IndexError, ValueError) as e:
        print(f"❌ Error parsing response: {e}")
        print(f"\nResponse received:")
        print(response.json())

Response status: 200


PREDICTION: 0

Result: Customer will NOT subscribe to term deposit

Full response:
{'success': True, 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:8a1e15cd-04c2-48aa-8f35-b4a8c11997d3:workspace:cd911947-f3e1-4596-adb4-8d87bd1060e9/ea09a650-52ef-4c52-841e-4f6fd0fce2be', 'prediction': [0], 'uuid': 'eec22cf2-44cd-431d-8b23-7de0ce9ff5d0'}, 'ReplicaID': 'banking-campaign-predictor-2158-3893-78f9fb4c96-g9pqx', 'Size': 259, 'StatusCode': 200}
